In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Joe\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Let's look at a computation graph as an example.

<img src="assets/computation_graph.png" alt="Drawing" style="width: 800px;"/>

In Tensorflow, it's implemented as follows:

In [2]:
a = tf.placeholder(tf.float32, None, name='a')
b = tf.placeholder(tf.float32, None, name='b')
c = tf.multiply(a,b)
d = tf.add(a, b)
e = tf.add(c, d)
f = tf.Print(e, [a, b, c, d, e], message='This is a, b, c, d, e')

In [3]:
a

<tf.Tensor 'a:0' shape=<unknown> dtype=float32>

In [4]:
with tf.Session() as sess:
    print(sess.run(e, feed_dict={a:1, b:2}))

5.0


In [6]:
with tf.Session() as sess:
    sess.run(f, feed_dict={a:1, b:2})

In [5]:
print(a)

Tensor("a:0", dtype=float32)


In [6]:
print(c)

Tensor("Mul:0", dtype=float32)


In [7]:
w = tf.Variable(0, dtype=tf.float32)
cost = tf.add(tf.add((w**2), tf.multiply(-10., w)), 25)

# tensorflow has overloaded arithmetic operatorars, so the following also works
# cost = w**2 - 10*w + 25

# Declare an optimizer to find the value for w that minimizes the cost function.
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

# Initialize all variablesSince we haven't run the optimizer so the value of the variable w is the value it's initialized to. After we run the optimizer once, we should see some change made to w's value.
init = tf.global_variables_initializer()

# Create a tensorflow session
sess = tf.Session()

# Eexecute a command
sess.run(init)
print(sess.run(w))

0.0


Since we haven't run the optimizer so the value of the variable w is the value it's initialized to. After we run the optimizer once, we should see some change made to w's value.

In [8]:
sess.run(train)
print(sess.run(w))

0.099999994


Apparently we need more epochs to let optimizer find the optimal value of w to minimize cost.

In [9]:
for i in range(1000):
    sess.run(train)
print(sess.run(w))
sess.close()

4.999988


In the above example, the cost function is fixed. It might be useful if we can change the coefficient. We can use tensorflow placeholder to achieve this. Tensorflow placeholder are empty containers with a certain shape. You can supply different value to a different session by using a feed dicionary. In fact, this will be the way we feed mini batches of training data to a neural network.

In [10]:
w = tf.Variable(0, dtype=tf.float32)
x = tf.placeholder(tf.float32, [3, 1])
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]

train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
print(sess.run(w))

0.0


In [11]:
coefficient = np.array([[1.], [-10.], [25.]])

sess.run(train, feed_dict={x:coefficient})
print(sess.run(w))

0.099999994


In [12]:
for i in range(1000):
    sess.run(train, feed_dict={x:coefficient})
print(sess.run(w))
sess.close()

4.999988


It's a good practice to use context manager to create and run a session.

In [13]:
w = tf.Variable(0, dtype=tf.float32)
x = tf.placeholder(tf.float32, [3, 1])
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train, feed_dict={x:coefficient})
        if i % 100 == 0:
            print("epoch: {:03d},   w : {:0.4f}".format(i, sess.run(w)))

epoch: 000,   w : 0.1000
epoch: 100,   w : 4.3502
epoch: 200,   w : 4.9138
epoch: 300,   w : 4.9886
epoch: 400,   w : 4.9985
epoch: 500,   w : 4.9998
epoch: 600,   w : 5.0000
epoch: 700,   w : 5.0000
epoch: 800,   w : 5.0000
epoch: 900,   w : 5.0000
